#**SCRAPING PHASE**

In [ ]:
import json 
import pandas as pd
import re
import string
import itertools as it
from itertools import combinations
from bs4 import BeautifulSoup
import requests 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Build up the url 
region = "lom"
prov = "MI"
idCom = "8042" # ID of Milan
pag = 1 
base_url = f"""https://www.immobiliare.it/api-next/search-list/real-estates/?fkRegione={region}&idProvincia={prov}&idComune={idCom}&idContratto=2&idCategoria=1&criterio=dataModifica&__lang=it&pag={pag}&paramsCount=1&path=%2Faffitto-case%2Fmilano%2F"""

immobiliare_raw = {}

def get_data(url):
  req = requests.get(url)
  data = json.loads(req.text)
  return data

immobiliare_raw[pag] = get_data(base_url) # first page

for page in range(2, 81): # we get the next pages
  url = f"https://www.immobiliare.it/api-next/search-list/real-estates/?fkRegione={region}&idProvincia={prov}&idComune={idCom}&idContratto=2&idCategoria=1&criterio=dataModifica&__lang=it&pag={page}&paramsCount=1&path=%2Faffitto-case%2Fmilano%2F"
  immobiliare_raw[page] = get_data(url)



In [ ]:
# The data are stored in a dictionary, with the pages of the site as keys. I create a list of all advertisements and extract the data that we need.

lista_annunci = []
for pagina in immobiliare_raw.keys():
  for annuncio in range(len(immobiliare_raw[pagina]['results'])):
    lista_annunci.append(immobiliare_raw[pagina]['results'][annuncio])

titles = []
urls = []
prices = []
properties = []
agency_urls = []
agency_types = []

for annuncio in range(len(lista_annunci)):

  titles.append(lista_annunci[annuncio]['seo']['title'])
  urls.append(lista_annunci[annuncio]['seo']['url'])

  if lista_annunci[annuncio]['realEstate']['price']['visible'] != False:
    prices.append(lista_annunci[annuncio]['realEstate']['price']['value'])  # We take only the ads with price showed
  else:
    prices.append(None)

  properties.append(lista_annunci[annuncio]['realEstate']['properties'])

  if 'agency' in lista_annunci[annuncio]['realEstate']['advertiser']:
    agency_urls.append(lista_annunci[annuncio]['realEstate']['advertiser']['agency']['agencyUrl']) # some ads have not agency, so they are managed by provates
    agency_types.append(lista_annunci[annuncio]['realEstate']['advertiser']['agency']['type'])
  else:
    agency_urls.append('announcement managed by a user')
    agency_types.append('private')


bathrooms = []
sizes = [] 
conditions = []
description = []
rooms = []
elevators = []
bedroomsns = []
floors = []
energies = []
features = []
photos = []

for annuncio in range(len(properties)):

  if 'bathrooms' in properties[annuncio][0]:
    bathrooms.append(properties[annuncio][0]['bathrooms']) 
  else:
    bathrooms.append('None')

  if 'surface' in properties[annuncio][0]:
    sizes.append(float(properties[annuncio][0]['surface'].replace(" ", "").replace("m²", "")))
  else:
    sizes.append(None)  

  if 'condition' in properties[annuncio][0]:
    conditions.append(properties[annuncio][0]['condition'])
  else:
    conditions.append(None)
   
  if 'description' in properties[annuncio][0]:
    description.append(properties[annuncio][0]['description'])
  else:
    description.append(None)

  if 'rooms' in properties[annuncio][0]:
    rooms.append(properties[annuncio][0]['rooms']) 
  else:
    rooms.append('None')

  elevators.append(properties[annuncio][0]['hasElevators'])

  if 'bedRoomsNumber' in properties[annuncio][0]:
    bedroomsns.append(int(properties[annuncio][0]['bedRoomsNumber']))
  else:
    bedroomsns.append(None) 

  if 'floor' in properties[annuncio][0]:
    floors.append(properties[annuncio][0]['floor']['abbreviation'])
  else:
    floors.append('None')

  if 'energy' in properties[annuncio][0]:
    energies.append(properties[annuncio][0]['energy'])
  else:
    energies.append(None)

  features.append(properties[annuncio][0]['features'])

  photos.append(properties[annuncio][0]['multimedia']['photos'])

In [ ]:
# The data are stored into a dataframe

immobiliare_toClean = pd.DataFrame({'Title' : titles, 'URL' : urls, 'Price/month' : prices, 'Agency URL' : agency_urls, 'If_agency' : agency_types, 
                                  'Bathrooms' : bathrooms, 'Size (m^2)' : sizes, 'Condition' : conditions, 'Description' : description, 'Locals' : rooms,
                                  'Elevator' : elevators, 'Bedrooms number' : bedroomsns, 'Floor' : floors, 'Energy info' : energies, 'Features' : features, 'Photos' : photos})


In [ ]:
#The data are stored into a json
immobiliare_toClean.to_json('/content/drive/MyDrive/Progetto Data Management/Data/immobiliare_toClean.json', orient='records')

#**DATA CLEANING PHASE**

In [ ]:
print(len(immobiliare_toClean)) # original lenght of the dataset

2000


###Null Data Removal and drop the special characters *\t*, *\n* and *\r* from the descriptions:

In [ ]:
# these are rows that we aren't able to clean, so we remove them
immobiliare_toClean = immobiliare_toClean[immobiliare_toClean['Floor'].notnull()]
immobiliare_toClean = immobiliare_toClean[immobiliare_toClean['Description'].notnull()]

In [ ]:
for index, row in immobiliare_toClean.iterrows():
    row['Description'] = re.sub(r'[\r\n\t]+', ' ', row['Description'])

In [ ]:
immobiliare_toClean = immobiliare_toClean.reset_index(drop=True)

###*Fit for Use*: search the street and district for every adv.

In [ ]:
# Import of street-to-district json file
with open('/content/drive/MyDrive/Progetto Data Management/Data/quartieri.json', 'r') as file:
    quartieri = json.load(file)

# import the custom functions
import sys
sys.path.append('/content/drive/MyDrive/Progetto Data Management/Code')
from functions import *

# get the ad's address
diz_address = addressFinder2(quartieri, immobiliare_toClean['Description'] + ' ' + immobiliare_toClean['Title'], [None] * len(immobiliare_toClean) )
immobiliare_toClean = immobiliare_toClean.reset_index()
immobiliare_toClean['Address'] = immobiliare_toClean['index'].astype(int).map(diz_address)
immobiliare_toClean = immobiliare_toClean.drop('index', axis=1) 
immobiliare_toClean = immobiliare_toClean[immobiliare_toClean['Address'].notnull()]


# get the ad's district
immobiliare_toClean["District"] = address_to_district(immobiliare_toClean, quartieri)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
pd.set_option('display.max_colwidth', None)
immobiliare_toClean[['Title', 'Description', 'Address']].head(50)

###Checking for Duplicates

In [ ]:
# Now, we are searching for "duplicated houses". Sometimes sites upload the same house multiple times, but with different listings.
# If two houses have the same value in some attributes (like price, agency running the ad, bathroom, floor etc) and have a too much similar description, we consider them as duplicated:

dups = immobiliare_toClean[immobiliare_toClean.duplicated(subset = ['Price/month', 'Agency URL', 'If_agency', 'Bathrooms', 
                                                                    'Size (m^2)', 'Locals', 'Elevator', 'Bedrooms number', 'Floor', 'District'], keep=False)]

groups = dups.groupby(by = ['Price/month','Agency URL', 'If_agency', 'Bathrooms', 'Size (m^2)', 'Locals', 'Elevator', 'Bedrooms number', 'Floor', 'District'], sort = False).groups

print(len(dups)) # number of possible duplicates

desc = dups['Description'].astype(str)

for i in desc.index:
  desc[i] = re.sub(r'[^\w]+', ' ', desc[i])

desc = desc.str.lower().str.split()

for i in desc.index: 
    desc[i] = set(desc[i])

lista = []

for val in groups.values():
    lista.append(desc.loc[val])
final_dup = []

for i in range(0, len(lista)):
  for x, y in it.combinations(lista[i], 2):
    if len(x & y)/len(x.union(y)) >= 0.9: # the threshold of description similarity is set to 0.9. That's because some agency use a standard text for their houses. 
                                          # Here the difference between descriptions is the adress and nothing else.
        final_dup.append(lista[i][lista[i] == x].index[0])
        final_dup.append(lista[i][lista[i] == y].index[0])



178


In [ ]:
print(len(immobiliare_toClean.iloc[list(set(final_dup))])) #number of duplicates with 0.9 as threshold

121


In [ ]:
immobiliare_toClean = immobiliare_toClean.drop(immobiliare_toClean.iloc[list(set(final_dup))].index).reset_index(drop=True)

###Adapting Data to an Appropriate Data Type

In [ ]:
for i in range(len(immobiliare_toClean)):

  # bathrooms
  if immobiliare_toClean.loc[i, 'Bathrooms'] == '3+':
    immobiliare_toClean.loc[i, 'Bathrooms'] = 3
  elif immobiliare_toClean.loc[i, 'Bathrooms'] == 'None':
    immobiliare_toClean.loc[i, 'Bathrooms'] = None
  else:
    try:
      immobiliare_toClean.loc[i, 'Bathrooms'] = int(immobiliare_toClean.loc[i, 'Bathrooms'])
    except:
      immobiliare_toClean.loc[i, 'Bathrooms'] = immobiliare_toClean.loc[i, 'Bathrooms']

  # locals
  if immobiliare_toClean.loc[i, 'Locals'] == '5+':
    immobiliare_toClean.loc[i, 'Locals'] = 5
  elif immobiliare_toClean.loc[i, 'Locals'] == 'None':
    immobiliare_toClean.loc[i, 'Locals'] = None
  else:
    try:
      immobiliare_toClean.loc[i, 'Locals'] = int(immobiliare_toClean.loc[i, 'Locals'])
    except:
      immobiliare_toClean.loc[i, 'Locals'] = immobiliare_toClean.loc[i, 'Locals'] 

  # floor
  if immobiliare_toClean.loc[i, 'Floor'] == '2 - 3':
    immobiliare_toClean.loc[i, 'Floor'] = 2
  elif immobiliare_toClean.loc[i, 'Floor'] == '3 - 4':
    immobiliare_toClean.loc[i, 'Floor'] = 3
  elif immobiliare_toClean.loc[i, 'Floor'] == '4 - 5':
    immobiliare_toClean.loc[i, 'Floor'] = 4
  elif immobiliare_toClean.loc[i, 'Floor'] == '5 - 6':
    immobiliare_toClean.loc[i, 'Floor'] = 5
  elif immobiliare_toClean.loc[i, 'Floor'] == '6 - 7':
    immobiliare_toClean.loc[i, 'Floor'] = 6
  elif immobiliare_toClean.loc[i, 'Floor'] == '7 - 8':
    immobiliare_toClean.loc[i, 'Floor'] = 7
  elif immobiliare_toClean.loc[i, 'Floor'] == '8 - 9':
    immobiliare_toClean.loc[i, 'Floor'] = 8  
  elif immobiliare_toClean.loc[i, 'Floor'] == 'R':
    immobiliare_toClean.loc[i, 'Floor'] = 0
  elif immobiliare_toClean.loc[i, 'Floor'] == 'T':
    immobiliare_toClean.loc[i, 'Floor'] = 0
  elif immobiliare_toClean.loc[i, 'Floor'] == 'T - R':
    immobiliare_toClean.loc[i, 'Floor'] = 0
  elif immobiliare_toClean.loc[i, 'Floor'] == 'S':
    immobiliare_toClean.loc[i, 'Floor'] = -1
  elif immobiliare_toClean.loc[i, 'Floor'] == 'S - T':
    immobiliare_toClean.loc[i, 'Floor'] = -1
  elif immobiliare_toClean.loc[i, 'Floor'] == 'None':
    immobiliare_toClean.loc[i, 'Floor'] = None
  else:
    try:
      immobiliare_toClean.loc[i, 'Locals'] = int(immobiliare_toClean.loc[i, 'Floor'])
    except:
      immobiliare_toClean.loc[i, 'Locals'] = immobiliare_toClean.loc[i, 'Floor']

###Outliers Removal

In [ ]:
immobiliare_toClean.describe()

,Price/month,Size (m^2),Bedrooms number
count,1843.000000,1656.000000,1794.000000
mean,2029.954422,65.167388,1.309922
std,2871.248966,40.412520,0.700443
min,70.000000,1.000000,0.000000
25%,1150.000000,45.000000,1.000000
50%,1490.000000,55.000000,1.000000
75%,2200.000000,75.000000,2.000000
max,77625.000000,450.000000,6.000000


In [ ]:
# - set a minimum of sqm (national disposition for a single local apartment)
# - set a minimum and maximum for price/month variable in order to
# drop the observations related to the short-rentals listings

outliers = (immobiliare_toClean[(immobiliare_toClean["Size (m^2)"] < 28) |
            (immobiliare_toClean["Price/month"] < 300)|
            (immobiliare_toClean["Price/month"] > 10000)]).index

len(outliers)

119

In [ ]:
immobiliare_Cleaned = immobiliare_toClean.drop(index=outliers).reset_index(drop=True)
len(immobiliare_Cleaned) # number of ads of the final dataset 

1739

In [ ]:
immobiliare_Cleaned[["Price/month", "Size (m^2)"]].describe()

,Price/month,Size (m^2)
count,1736.000000,1537.000000
mean,1922.322005,67.326392
std,1306.520237,35.861603
min,600.000000,28.000000
25%,1170.000000,50.000000
50%,1500.000000,55.000000
75%,2200.000000,75.000000
max,10000.000000,400.000000


#**DATA STORAGE PHASE**

In [ ]:
#The FINAL data are stored into a json, ready for data integration
with open("/content/drive/MyDrive/Progetto Data Management/Data/immobiliare.json", "w") as file:
  json.dump(immobiliare_Cleaned.to_dict(orient="index"), file)